In [30]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError

In [31]:
uri = 'bolt://localhost:7687'
music_dict, user_dict, word_dic, tags, words = {}, {}, {}, set(), set()
wordnet, dbpedia_type, dbpedia_subject = {}, {}, {}

count = 0
driver = GraphDatabase.driver(uri, auth=("yexiao", "yexiao"))

def add_to_word(dic, s, p, o):
    if not dic.get(s):
        dic.setdefault(s, {p: o})
    else:
        dic[s].setdefault(p, o)
    return 

def add_to_lst(dic, s, p, o):
    if not dic.get(s):
        dic.setdefault(s, [ob])
    else:
        dic[s].append(ob)
    return p

with open('kg_triples.txt', 'r', encoding='UTF-8') as kg_graph:
    for line in kg_graph:
#         if count >= 300000:
#             break
        curr = line.strip('\n').split('\t')
        subject, predicate, ob = curr[0], curr[1], curr[2]
        if predicate == "listened":
            #append the user information
            if not user_dict.get(subject):
                user_dict.setdefault(subject, [ob])
            else:
                user_dict[subject].append(ob)
        elif predicate in ["hasKeyword", "hasTag"]:
            #append the music information
            tags.add(ob) if predicate == "hasTag" else words.add(ob)
            temp = {"hasKeyword": None, "hasTag": None}
            if not music_dict.get(subject):
                temp[predicate] = ob
                music_dict.setdefault(subject, temp)
            else:
                music_dict[subject].setdefault(predicate, ob)
        elif predicate == "synset_member":
                wordnet.setdefault(ob, {})
                add_to_word(word_dic, subject, predicate, ob)
        elif predicate == "relation":
                dbpedia_type.setdefault(ob, {})
                add_to_word(word_dic, subject, predicate, ob)
        elif predicate == "hypernym":
             add_to_word(wordnet, subject, predicate, ob)
        elif predicate == "subject": # now subject and broader
            dbpedia_subject.setdefault(ob, [])
            add_to_lst(dbpedia_type, subject, predicate, ob)
        else:
            add_to_lst(dbpedia_subject, subject, predicate, ob)
        count += 1        

    

In [ ]:
#write querys
# dic_lst = list(music_dict.items())
count = 0
with driver.session() as session:
    #clear db
    session.run("MATCH (n) DETACH DELETE n")
    #write wordnet type
    for key in wordnet.keys():
        create_query = "CREATE (a:WordNet_word { label: $name})"
        session.run(create_query, name=key)
    for key, value in wordnet.items():
        for obj in value.values():
            match_query = "MATCH (a:WordNet_word), (b:WordNet_word) \
                WHERE a.label=$name_one AND b.label=$name_two \
                CREATE (a) -[:hyperonym]-> (b)"
            session.run(match_query, name_one=key, name_two=obj)
            
    #write dbpedia subject
    for key in dbpedia_subject.keys():
        create_query = "CREATE (a:Dbpedia_subject {label:$name})"
        session.run(create_query, name=key)
    
    for key, value in dbpedia_subject.items():
        # {key, [obj]}
        for obj in value:
            match_query = "MATCH (a:Dbpedia_subject), (b:Dbpedia_subject) \
                WHERE a.label=$name_one AND b.label=$name_two \
                CREATE (a) -[:broader]-> (b)"
            session.run(match_query, name_one=key, name_two=obj)
            
    #write dbpedia type
    for key in dbpedia_type.keys():
        create_query = "CREATE (a:Dbpedia_type {label:$name})"
        session.run(create_query, name=key)
    
    for key, value in dbpedia_type.items():
        for obj in value:
            match_query = "MATCH (a:Dbpedia_type), (b:Dbpedia_subject) \
                WHERE a.label=$name_one AND b.label=$name_two \
                CREATE (a) -[:subject]-> (b)"
            session.run(match_query, name_one=key, name_two=obj)
    
    for key, value in word_dic.items():
        temp_dic = {}
        temp_dic.update(value)
        if key in tags:
            temp_dic.setdefault("tag", key)
            create_query = "CREATE (a:Tag { label: $tag, relation: $relation, synset_member: $synset_member})"
            session.run(create_query, tag=temp_dic.get('tag'), relation=temp_dic.get('relation') if temp_dic.get(
            'relation') else None, synset_member=temp_dic.get('synset_member') if temp_dic.get('synset_member') else None)
            #match for the outer graph
            if temp_dic.get('synset_member'):
                match_query = "MATCH (a:Tag), (b:WordNet_word) \
                    WHERE a.label = $name_id AND b.label = $word \
                    CREATE (a)-[:synset_member]->(b)"
                session.run(match_query, name_id=key, word=temp_dic.get('synset_member'))
            if temp_dic.get('relation'):
                match_query = "MATCH (a:Tag), (b:Dbpedia_type) \
                    WHERE a.label = $name_id AND b.label = $name \
                    CREATE (a)-[:relation]->(b)"
                session.run(match_query, name_id=key, name=temp_dic.get('relation'))
                
        elif key in words:
            temp_dic.setdefault("word",key)
            create_query = "CREATE (a:Word { label: $word, relation: $relation, synset_member: $synset_member})"
            session.run(create_query, word=temp_dic.get('word'), relation=temp_dic.get('relation') if temp_dic.get(
            'relation') else None, synset_member=temp_dic.get('synset_member') if temp_dic.get('synset_member') else None)
            #match for the outer graph
            if temp_dic.get('synset_member'):
                match_query = "MATCH (a:Word), (b:WordNet_word) \
                    WHERE a.label = $name_id AND b.label = $word \
                    CREATE (a)-[:synset_member]->(b)"
                session.run(match_query, name_id=key, word=temp_dic.get('synset_member'))
            if temp_dic.get('relation'):
                match_query = "MATCH (a:Word), (b:Dbpedia_type) \
                    WHERE a.label = $name_id AND b.label = $name \
                    CREATE (a)-[:relation]->(b)"
                session.run(match_query, name_id=key, name=temp_dic.get('relation'))            
        
    for key, value in music_dict.items():
        temp_dic = {}
        temp_dic.setdefault("id", key)
        temp_dic.update(value)
        create_query = "CREATE (a:Music { label: $id, hasKeyword: $hasKeyword, hasTag: $hasTag})"
        session.run(create_query, temp_dic)
        if temp_dic.get('hasKeyword'):
            match_query = "MATCH (a:Music), (b:Word) \
                WHERE a.label = $name_id AND b.label = $word \
                CREATE (a)-[:hasKeyword]->(b)"
            session.run(match_query, name_id=key, word=temp_dic.get('hasKeyword'))
        if temp_dic.get('hasTag'):
            match_query = "MATCH (a:Music), (b:Tag) \
                WHERE a.label = $name_id AND b.label = $tag \
                CREATE (a)-[:hasTag]->(b)"
            session.run(match_query, name_id=key, tag=temp_dic.get('hasTag'))
            
    for user_id, dic in user_dict.items():
        #the query might vary when further graph introduced
        create_query = "CREATE (a:Person { label: $name_id })"
        session.run(create_query, name_id=user_id)
        for music in dic:
            match_query = "MATCH (a:Person), (b:Music) \
            WHERE a.label = $name_id AND b.label = $music_id \
            CREATE (a)-[:listened]->(b)"
            session.run(match_query, name_id=user_id, music_id=music)
            